# RSI底背离信号后收益率分析

本Notebook将分析RSI底背离信号出现后，股票在未来一段时间的收益率表现。

In [1]:
import sys
sys.path.append('.')

import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 设置tushare token
TUSHARE_TOKEN = os.getenv('TUSHARE_TOKEN')
if not TUSHARE_TOKEN:
    raise ValueError('请设置TUSHARE_TOKEN环境变量')

In [2]:
# 导入自定义模块
try:
    import tushare as ts
    from signals.returns import calculate_future_returns
    ts.set_token(TUSHARE_TOKEN)
    pro = ts.pro_api()
except ImportError as e:
    print(f'导入模块时出错: {e}')
    exit(1)

In [3]:
def analyze_returns_after_signals(signals_file, days=5):
    """
    分析RSI底背离信号后的收益率
    """
    # 读取信号文件
    signals_df = pd.read_csv(signals_file)
    
    # 获取所有股票代码
    stock_codes = signals_df['stock_code'].unique()
    
    # 计算未来收益率
    returns_data = calculate_future_returns(stock_codes.tolist(), days)
    
    # 将收益率数据添加到信号数据中
    signals_df['future_return'] = signals_df['stock_code'].map(lambda x: returns_data.get(x, {}).get('return_pct', np.nan) if returns_data.get(x, {}).get('error') is None else np.nan)
    
    # 计算平均收益率
    avg_return = signals_df['future_return'].mean()
    
    return signals_df, avg_return

In [4]:
# 分析收益率
signals_file = 'multiple_stocks_rsi_bullish_divergence_20250828.csv'
results_df, avg_return = analyze_returns_after_signals(signals_file, days=5)

print(f'分析的信号文件: {signals_file}')
print(f'平均收益率: {avg_return:.2f}%')

# 显示前几行数据
print('收益率分析结果:')
print(results_df.head(20))

# 保存结果
output_file = f'rsi_divergence_returns_analysis_{datetime.now().strftime("%Y%m%d")}.csv'
results_df.to_csv(output_file, index=False)
print(f'结果已保存到 {output_file}')

分析的信号文件: multiple_stocks_rsi_bullish_divergence_20250828.csv
平均收益率: -2.75%
收益率分析结果:
   stock_code stock_name        date  close_price  rsi_value  future_return
0   600519.SH       贵州茅台  2024-11-15      1505.50  48.864572      -2.840311
1   600519.SH       贵州茅台  2025-01-22      1413.31  37.288323      -2.840311
2   600519.SH       贵州茅台  2025-01-24      1408.40  48.626354      -2.840311
3   600519.SH       贵州茅台  2025-01-27      1407.41  48.223844      -2.840311
4   600519.SH       贵州茅台  2025-02-05      1376.82  38.976652      -2.840311
5   600519.SH       贵州茅台  2025-02-06      1385.69  41.491257      -2.840311
6   600519.SH       贵州茅台  2025-02-10      1404.00  46.798597      -2.840311
7   600519.SH       贵州茅台  2025-02-11      1390.75  34.840320      -2.840311
8   600519.SH       贵州茅台  2025-02-12      1415.27  43.053955      -2.840311
9   600519.SH       贵州茅台  2025-02-17      1443.34  49.142857      -2.840311
10  600519.SH       贵州茅台  2025-04-24      1522.42  45.952653      -2.840311
11  

In [5]:
# 统计分析
positive_returns = results_df[results_df['future_return'] > 0]
negative_returns = results_df[results_df['future_return'] < 0]

print(f'总信号数: {len(results_df)}')
print(f'正收益信号数: {len(positive_returns)} ({len(positive_returns)/len(results_df)*100:.2f}%)')
print(f'负收益信号数: {len(negative_returns)} ({len(negative_returns)/len(results_df)*100:.2f}%)')

# 按股票统计
stock_stats = results_df.groupby('stock_name')['future_return'].agg(['count', 'mean', 'std']).round(2)
stock_stats.columns = ['信号数', '平均收益率(%)', '收益率标准差(%)']
print('按股票统计的收益率:')
print(stock_stats)

总信号数: 77
正收益信号数: 0 (0.00%)
负收益信号数: 77 (100.00%)
按股票统计的收益率:
            信号数  平均收益率(%)  收益率标准差(%)
stock_name                          
东方财富         13     -3.61        0.0
中兴通讯         17     -3.80        0.0
中芯国际         27     -1.61        0.0
贵州茅台         20     -2.84        0.0
